In [ ]:
import pandas as pd
import os
from project.dataset import Dataset, VALDODataset
from project.preprocessing import NiftiToTensorTransform, get_transform
from project.utils import collatev2, compute_statistics
from torch.utils.data import DataLoader
import torch
from project.model import VisionTransformer, ISAVIT
from torch.optim import Adam
import torch.nn as nn
import torch.nn.functional as F
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from project.model.feeder import Feeder
import seaborn as sns
from project.evaluation import isa_vit_metric, Tracker

In [ ]:
tk = Tracker()

In [ ]:
import logging
from datetime import datetime as dtt
import os

path = 'logs'
os.makedirs(path, exist_ok=True)
os.makedirs('history', exist_ok=True)
rn = dtt.now()
dte = rn.strftime('%b_%d_%Y_%H%M%S')

logger = logging.getLogger('andy')
fh = logging.FileHandler(f'logs/{dte}.log')
formatter = logging.Formatter(
    '%(asctime)s - %(levelname)s - %(message)s'
)

logger.setLevel(logging.DEBUG)
fh.setLevel(logging.DEBUG)
fh.setFormatter(formatter)

logger.addHandler(fh)

tk.date = rn
tk.logfile = f'{dte}.log'

dte

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tk.device = device
device

### Config for fitter


In [ ]:
image_size = 300
patch_size = 16

config = {
    'model': ISAVIT(
        d_model=512,
        patch_size=patch_size,
        dim_ff=1600
    ).to(device),
    'optimizer': torch.optim.Adam,
    'device': device,
    'epochs': 5,
    'loss': nn.BCEWithLogitsLoss(),
    # 'loss': nn.MSELoss(),
    'lr': 0.0001
}

tk.model = 'ViT'
tk.model_hyperparams = config['model'].config
tk.optimizer = f"{config['optimizer']}"
tk.epochs = config['epochs']
tk.loss = f"{config['loss']}"
tk.lr = config['lr']

### Create Stage 1 Network


In [ ]:
from project.model import Feeder, RPN, GCRPN

resize = get_transform(
    height=patch_size,
    width=patch_size,
    p=1.0,
    rpn_mode=False
)

feeder = Feeder(resize)
rpn = RPN(
    input_dim=512,
    output_dim=4,
    image_size=image_size,
    nh=4
)

tk.uses_resnet = rpn.config['resnet']

stone = GCRPN(
    rpn=rpn,
    feeder=feeder,
    image_size=image_size,
    patch_size=patch_size
).to(device)

In [ ]:
stow = 'rpn_kfold/Nov_17_2024_092857/RPN_Nov_17_2024_092857_fold_1.pt'
tk.stage1_weights = stow
stone.rpn.load_state_dict(torch.load(stow))

#### Load ViT Weights


### Load dataset


In [ ]:
ds = Dataset()

data = pd.read_csv('targets.csv')
data.shape

In [ ]:
data = data.query('has_microbleed_slice == 1').reset_index(drop=True)
tk.only_cmb_slices = True
data

### `DataLoader` Generator


In [ ]:
def iqr(data, col):
    q3 = data[col].quantile(0.75)
    q1 = data[col].quantile(0.25)
    iqr = q3-q1
    new = data[(data[col] < (q3 + 1.5*iqr)) & (data[col] > (q1 - 1.5*iqr))]
    return new

In [ ]:
from sklearn.model_selection import train_test_split

def make_loaders(data,
                 cohort,
                 batch_size,
                 test_size=0.2,
                 random_state=12,
                 target_shape=(300, 300),
                 rpn_mode=True,
                 logger=None,
                 tracker=tk
                ):
    if cohort == 1:
        tracker.cohort1 = True
    if cohort == 2:
        tracker.cohort2 = True
    if cohort == 3:
        tracker.cohort3 = True
    tracker.batch_size = batch_size
    tracker.test_size = test_size
    tracker.target_shape = target_shape
    data = data[data.cohort == cohort]
    # data = iqr(data, 'max_value')
    
    s = f'Creating loaders for Cohort {cohort}\n'

    data_train, data_test = train_test_split(
        data,
        test_size=test_size,
        random_state=random_state
    )

    s += f'TRAIN & TEST: {data_train.shape, data_test.shape}\n'

    paths = data_train.stripped.unique().tolist()
    s += f'Total Unique MRI Samples in data_train: {len(paths)}\n'
    
    global_min, global_max = compute_statistics(paths)
    s += f'GLOBAL MIN & MAX {global_min, global_max}\n'

    transform = NiftiToTensorTransform(
        target_shape=target_shape,
        rpn_mode=rpn_mode,
        normalization=(global_min, global_max)
    )

    train_set = VALDODataset(
        cases=data_train.stripped.tolist(),
        masks=data_train.masks.tolist(),
        target=data_train.target.tolist(),
        transform=transform
    )
    val_set = VALDODataset(
        cases=data_test.stripped.tolist(),
        masks=data_test.masks.tolist(),
        target=data_test.target.tolist(),
        transform=transform
    )

    train_loader = DataLoader(
        train_set,
        shuffle=True,
        batch_size=batch_size,
        collate_fn=collatev2
    )
    val_loader = DataLoader(
        val_set,
        shuffle=True,
        batch_size=batch_size,
        collate_fn=collatev2
    )

    if logger != None:
        logger.info(s)
    else:
        print(s)
    
    return train_loader, val_loader, train_set, val_set

### Fitter


In [ ]:
from project import Fitter

class ViTFitter(Fitter):
    
    def fit(self, train_loader, val_loader, stage1):
        train_history = []
        val_history = []
        train_metric_history = []
        val_metric_history = []
        for epoch in range(self.epochs):
            self.log(f'EPOCH {epoch} ==============================')
            train_loss, train_metric = self.train_one_epoch(train_loader, stage1)
            val_loss, val_metric = self.validation(val_loader, stage1)
            train_history.append(train_loss)
            val_history.append(val_loss)
            train_metric_history.append(train_metric)
            val_metric_history.append(val_metric)
        return train_history, val_history, train_metric_history, val_metric_history

    def train_one_epoch(self, train_loader, stage1):
        self.model.train()
        loss_history = []
        evaluation_metric = {
            'dice_score': [], 
            'precision_score': [], 
            'recall_score': [], 
            'f1_score': [],
            'fpr': []
        }
        counter = 0
        for batch in train_loader:
            Y = []
            T = []
            for slices, masks, target, case in batch:
                slices = slices.squeeze(1).float().to(self.device)
                masks = masks.float().to(self.device)

                with torch.inference_mode():
                    x, t = stage1(slices, masks, target)
                
                # self.log(f'{x.requires_grad}, {t.requires_grad}')
                # self.log(f'{x.shape}, {t.shape}')

                x = x.flatten(2).float().to(self.device)
                t = t.flatten(2).float().to(self.device)
                # self.log(f'XT SHAPES: {x.shape}, {t.shape}')
                
                y = self.model(x, target)
                
                # print('Prediction:', (y.sigmoid() >= 0.5).int().unique())
                # print('Truth:', (t[target] >= 0).int().unique())

                dice_score, precision_score, recall_score, f1_score, fpr = isa_vit_metric((y.sigmoid().squeeze().detach().cpu().numpy() >= 0.5), (t[target].squeeze().detach().cpu().numpy() > 0))

                evaluation_metric['dice_score'].append(dice_score)
                evaluation_metric['precision_score'].append(precision_score)
                evaluation_metric['recall_score'].append(recall_score)
                evaluation_metric['f1_score'].append(f1_score)
                evaluation_metric['fpr'].append(fpr)
                # self.log(f'EVAL METS: {dice_score, precision_score, recall_score, f1_score, fpr}')

                Y.append(y)
                T.append(t[target])
            
            losses = self.loss(torch.stack(Y), torch.stack(T))
            self.optimizer.zero_grad()
            losses.backward()
            self.optimizer.step()
            counter += 1
            self.log(f'Batch:\t{counter}/{len(train_loader)}')
            self.log(f'Batch samples:\t{len(batch)}')
            self.log(f'Current error:\t{losses}\n')
            
            loss_history.append(losses.detach().cpu().numpy())
        
        self.log(f'\nTraining Evaluation Metric:')
        self.log(f"Avg Dice: {sum(evaluation_metric['dice_score']) / len(evaluation_metric['dice_score'])}")
        self.log(f"Avg Precision: {sum(evaluation_metric['precision_score']) / len(evaluation_metric['precision_score'])}")
        self.log(f"Avg Recall: {sum(evaluation_metric['recall_score']) / len(evaluation_metric['recall_score'])}")
        self.log(f"Avg F1: {sum(evaluation_metric['f1_score']) / len(evaluation_metric['f1_score'])}")
        self.log(f"Avg FPR: {sum(evaluation_metric['fpr']) / len(evaluation_metric['fpr'])}\n")
        return loss_history, evaluation_metric
    
    def validation(self, val_loader, stage1):
        self.model.eval()
        loss_history = []
        evaluation_metric = {
            'dice_score': [], 
            'precision_score': [], 
            'recall_score': [], 
            'f1_score': [],
            'fpr': []
        }
        with torch.inference_mode():
            for batch in val_loader:
                Y = []
                T = []
                for slices, masks, target, case in batch:
                    slices = slices.squeeze(1).float().to(self.device)
                    masks = masks.float().to(self.device)
                    x, t = stage1(slices, masks, target)
                    x = x.flatten(2).float().to(self.device)
                    t = t.flatten(2).float().to(self.device)
                    y = self.model(x, target)

                    dice_score, precision_score, recall_score, f1_score, fpr = isa_vit_metric((y.sigmoid().squeeze().detach().cpu().numpy() >= 0.5), (t[target].squeeze().detach().cpu().numpy() > 0))
                    evaluation_metric['dice_score'].append(dice_score)
                    evaluation_metric['precision_score'].append(precision_score)
                    evaluation_metric['recall_score'].append(recall_score)
                    evaluation_metric['f1_score'].append(f1_score)
                    evaluation_metric['fpr'].append(fpr)
                    Y.append(y)
                    T.append(t[target])
                
                losses = self.loss(torch.stack(Y), torch.stack(T))
                loss_history.append(losses.cpu().numpy())
        
        self.log(f'\nValidations Evaluation Metric:')
        self.log(f"Avg Dice: {sum(evaluation_metric['dice_score']) / len(evaluation_metric['dice_score'])}")
        self.log(f"Avg Precision: {sum(evaluation_metric['precision_score']) / len(evaluation_metric['precision_score'])}")
        self.log(f"Avg Recall: {sum(evaluation_metric['recall_score']) / len(evaluation_metric['recall_score'])}")
        self.log(f"Avg F1: {sum(evaluation_metric['f1_score']) / len(evaluation_metric['f1_score'])}")
        self.log(f"Avg FPR: {sum(evaluation_metric['fpr']) / len(evaluation_metric['fpr'])}\n")
        return loss_history, evaluation_metric

### Train-Test Split


In [ ]:
_, _, t1, v1 = make_loaders(
    data=data,
    cohort=1,
    batch_size=20,
    rpn_mode=False
)

_, _, t3, v3 = make_loaders(
    data=data,
    cohort=3,
    batch_size=20,
    rpn_mode=False
)

In [ ]:
from torch.utils.data import ConcatDataset

train_set = ConcatDataset([t1, t3])
test_set = ConcatDataset([v1, v3])

In [ ]:
len(train_set)

In [ ]:
len(test_set)

### Training


In [ ]:
from torch.utils.data import Subset
from sklearn.model_selection import KFold

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=45)
kf

In [ ]:
fold_metrics = []
vit_kfold_dir = f'./vit_kfold/{dte}'
os.makedirs(vit_kfold_dir, exist_ok=True)

for i, (train_index, val_index) in enumerate(kf.split(train_set)):
    print(f'Fold {i + 1}/{5}')
    
    config['model'] = ISAVIT(
        d_model=512,
        patch_size=patch_size,
        dim_ff=1600
    ).to(device)
    
    fitter = ViTFitter(config, logger=logger)
    
    train_subset = Subset(train_set, train_index)
    val_subset = Subset(train_set, val_index)
    
    train_subset_dl = DataLoader(
        train_subset,
        shuffle=True,
        batch_size=20,
        collate_fn=collatev2
    )
    
    val_subset_dl = DataLoader(
        val_subset,
        shuffle=True,
        batch_size=20,
        collate_fn=collatev2
    )
    
    thist, vhist, tmhist, vmhist = fitter.fit(train_subset_dl, val_subset_dl, stone)
    
    fold_metrics.append({
        'fold': i + 1,
        'training_history': thist,
        'validation_history': vhist,
        'training_metrics': tmhist,
        'validation_metrics': vmhist
    })
    
    torch.save(fitter.model.state_dict(), f'{vit_kfold_dir}/ViT_{dte}_fold_{i+1}.pt')

    

In [ ]:
import winsound

winsound.Beep(500, 500)
winsound.Beep(500, 500)
winsound.Beep(500, 500)

In [ ]:
import numpy as np 
[np.array(fold).mean() for fold in fold_metrics[i]['training_history']]

In [ ]:
fold_metrics[0]

thist_array = []
vhist_array = []
v_dice_array = []
v_precision_array = []
v_recall_array = []
v_f1score_array = []
v_fpr_array = []


def get_avg_score(set, metric):
    return np.array(fold_metrics[i][set][0][metric]).mean()

for i in range(5):
    print(f'K-Fold {i+1}')
    
    thist = [np.array(fold).mean() for fold in fold_metrics[i]['training_history']]
    print(f'Training History: {thist}')
    thist_array.append(thist)
    
    vhist = [np.array(fold).mean() for fold in fold_metrics[i]['validation_history']]
    print(f'Validation History: {vhist}')
    vhist_array.append(vhist)
    
    # print('\nTraining Metrics')
    # print(f"Average IOU: {get_avg_score('training_metrics', 'iou_score')}")
    # print(f"Average Precision: {get_avg_score('training_metrics', 'precision_score')}")
    # print(f"Average Recall: {get_avg_score('training_metrics', 'recall_score')}")
    # print(f"Average F1 Score: {get_avg_score('training_metrics', 'f1_score')}")
    
    print(f'\nFold {i+1}: Validation Metrics')
    v_dice = get_avg_score('validation_metrics', 'dice_score')
    print(f"Average Dice Score: {v_dice}")
    v_dice_array.append(v_dice)
    
    v_precision = get_avg_score('validation_metrics', 'precision_score')
    print(f"Average Precision: {v_precision}")
    v_precision_array.append(v_precision)
    
    v_recall = get_avg_score('validation_metrics', 'recall_score')
    print(f"Average Recall: {v_recall}")
    v_recall_array.append(v_recall)
    
    v_f1 = get_avg_score('validation_metrics', 'f1_score')
    print(f"Average F1 Score: {v_f1}")
    v_f1score_array.append(v_f1)
    
    v_f1 = get_avg_score('validation_metrics', 'f1_score')
    print(f"Average F1 Score: {v_f1}")
    v_f1score_array.append(v_f1)
    
    v_fpr = get_avg_score('validation_metrics', 'fpr')
    print(f"Average False Positive Rate: {v_fpr}")
    v_fpr_array.append(v_fpr)
    
    print('\n===================================\n')

In [ ]:
print('K-Fold Cross Validation: Validation Metrics')
print(f'Validation - Average Dice Score: {np.array(v_dice_array).mean()} ')
print(f'Validation - Average Precision: {np.array(v_precision_array).mean()} ')
print(f'Validation - Average Recall: {np.array(v_recall_array).mean()} ')
print(f'Validation - Average F1 Score: {np.array(v_f1score_array).mean()} ')
print(f'Validation - Average False Positive Rate: {np.array(v_fpr_array).mean()} ')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

fig, axes = plt.subplots(5,1, figsize=(5, 15))

for i in range(5):
    th = np.array(thist_array[i])
    vh = np.array(vhist_array[i])
    
    ax=axes[i]
    
    sns.lineplot(th, label='Training history', ax=ax)
    sns.lineplot(vh, label='Validation history', ax=ax)
    
    ax.set_title(f'Training and Validation History: Fold {i+1}')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Loss')
    ax.legend()
  
plt.tight_layout()  
plt.show()

In [ ]:
sth = f'history/{dte}_thist.pt'
svh = f'history/{dte}_vhist.pt'
tk.saved_thist = sth
tk.saved_vhist = svh
torch.save(thist_array, sth)
torch.save(vhist_array, svh)

### Evaluation


In [ ]:
test_dataloader = DataLoader(
    test_set,
    shuffle=True,
    batch_size=20,
    collate_fn=collatev2
)

In [ ]:
print('K-Fold Cross Validation: Validation Metrics')
print(f'Validation - Average Dice Score: {np.array(v_dice_array).mean()} ')
print(f'Validation - Average Precision: {np.array(v_precision_array).mean()} ')
print(f'Validation - Average Recall: {np.array(v_recall_array).mean()} ')
print(f'Validation - Average F1 Score: {np.array(v_f1score_array).mean()} ')
print(f'Validation - Average False Positive Rate: {np.array(v_fpr_array).mean()} ')

In [ ]:
test_dice = []
test_precision = []
test_recall = []
test_f1 = []
test_fpr = []

for filename in os.listdir(vit_kfold_dir):
    fitter.model.load_state_dict(torch.load(f'{vit_kfold_dir}/{filename}'))
    
    h, mh = fitter.validation(test_dataloader)
    valmets = pd.DataFrame(mh)
    mets = valmets.mean()
    
    print(f'Version {filename}:')
    
    test_dice.append(mets.dice_score)
    test_precision.append(mets.precision_score)
    test_recall.append(mets.recall_score)
    test_f1.append(mets.f1_score)
    test_fpr.append(mets.fpr)
    
    print('\n========================\n')

In [ ]:
dice_score = np.array(test_dice).mean()
precision_score = np.array(test_precision).mean()
recall_score = np.array(test_recall).mean()
f1_score = np.array(test_f1).mean()
fpr = np.array(test_fpr).mean()

In [ ]:
print('Test Set Performance Metrics')
print(f'Average Dice Score: {dice_score} ')
print(f'Average Precision: {precision_score} ')
print(f'Average Recall: {recall_score} ')
print(f'Average F1 Score: {f1_score} ')
print(f'Average FPR: {fpr} ')

In [ ]:
tk.dice = dice_score
tk.precision = precision_score
tk.recall = recall_score
tk.f1 = f1_score
tk.fpr = fpr

### Save the weights


In [ ]:
s = [file for file in os.listdir(vit_kfold_dir)]
s

In [ ]:
tk.saved_weights = s

### Trial


In [ ]:
sample = next(enumerate(test_dataloader))

In [ ]:
case = sample[1][0]

In [ ]:
slices, masks, target, path = case

In [ ]:
sns.heatmap(masks[target].squeeze())

In [ ]:
slices = slices.squeeze(1).float().to(device)
masks = masks.float().to(device)

In [ ]:
x, t = stone(slices, masks, target)

In [ ]:
f, a = plt.subplots(1, 2, figsize=(10, 4))

f.tight_layout()
ax1 = a.flat[0]
ax2 = a.flat[1]
ax1.set_title('MRI Slice Crop')
ax2.set_title('Mask Slice Crop')
sns.heatmap(x[target].squeeze(), ax=ax1)
sns.heatmap(t[target].squeeze(), ax=ax2)

In [ ]:
vit = config['model']

In [ ]:
x.shape

In [ ]:
y = vit(x.flatten(2).to(device), target)

In [ ]:
y = y.view(patch_size, patch_size)

In [ ]:
f, a = plt.subplots(1, 2, figsize=(10, 4))

f.tight_layout()
ax1 = a.flat[0]
ax2 = a.flat[1]
ax1.set_title('Mask Prediction')
ax2.set_title('Mask Truth')
sns.heatmap((y > -0.3).detach().cpu(), ax=ax1, vmax=1)
sns.heatmap(t[target].squeeze(), ax=ax2, vmax=1)

# Log Progress


In [ ]:
tk.notes = '''
- 5 fold cross validation for isa-vit
- fixed the reinitialization issue of the fitter
- save weights of each fold
- evaluate test data using each fold
- need to imrpove this by implementing end to end in one notebook (rpn and vit)
'''

In [ ]:
tk()

In [ ]:
if os.path.exists('history/runs.csv'):
    print('Merging to old df')
    prev_df = pd.read_csv('history/runs.csv', index_col='date')
    merged = pd.concat([prev_df, tk()])
    merged.to_csv('history/runs.csv')
else:
    print('Making new csv file')
    tk().to_csv('history/runs.csv')